# Read NetCDF file from MeteoSwiss

In [39]:
import netCDF4 as nc
import numpy as np
import os
import rasterio
from datetime import datetime, timedelta
from rasterio.transform import from_origin

In [33]:
fname = 'CH2018_pr_CLMCOM-CCLM4_MPIESM_EUR44_RCP45_QMgrid_1981-2099.nc'
outdir = 'CH2018_pr_CLMCOM-CCLM4_MPIESM_EUR44_RCP45_QMgrid_1981-2099'

In [21]:
ds = nc.Dataset(fname)

Get all variable keys:

In [22]:
k = ds.variables.keys()

In [5]:
k

dict_keys(['lon', 'lat', 'time', 'pr'])

Read the information about the time variable:

In [23]:
ds.variables['time']

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    long_name: time
    units: days since 1900-1-1 00:00:00
    calendar: standard
    axis: T
unlimited dimensions: time
current shape = (43464,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [44]:
t = ds.variables['time'][:]
t

masked_array(data=[29585.5, 29586.5, 29587.5, ...,     0. ,     0. ,
                       0. ],
             mask=False,
       fill_value=1e+20)

In [45]:
np.min(t), np.max(t)

(0.0, 34801.5)

In [46]:
dt_max = datetime(1900, 1, 1) + timedelta(days=np.max(t))
dt_max

datetime.datetime(1995, 4, 14, 12, 0)

Maximum date is in 1995! This cannot be correct. All other values seem to be 0.
Probably, we just need to increase the day by 1 each time.

In [24]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: CH2018 bias-corrected and downscaled data
    project: CH2018 - New Climate Scenarios for Switzerland (http://www.climate-scenarios.ch)
    institution: The CH2018 Consortium (MeteoSwiss, ETH Zurich, C2SM, University of Bern, scnat) and the National Centre for Climate Services NCCS (http://www.nccs.ch)
    creator_name: Sven Kotlarski (MeteoSwiss), sven.kotlarski@meteoswiss.ch
    contact: klimaszenarien@meteoschweiz.ch
    license: https://creativecommons.org/licenses/by/4.0
    frequency: day
    spatial_scale: regular 2km grid
    CORDEX_RCM: CLMCOM-CCLM4
    CORDEX_GCM: MPIESM
    CORDEX_DOMAIN: EUR44
    CORDEX_SCENARIO: RCP45
    version: 1.0.0
    creation_date: Wed Nov  7 18:07:56 CET 2018
    references: [1] CH2018 (2018) CH2018 - Climate Scenarios for Switzerland, Technical Report, National Centre for Climate Services, Zurich,  ISBN: 978-3-9525031-4-0, Chapter 5. [2] Fe

In [25]:
lon = ds.variables['lon'][:]

In [27]:
lon[1] - lon[0]

0.020833333333333037

In [29]:
lat = ds.variables['lat'][:]

In [30]:
lat[1] - lat[0]

0.020833333333328596

In [31]:
len(lat), len(lon)

(101, 220)

Définir une fonction qui convertit le NetCDF en GeoTIFF:

In [50]:
def netcdf_to_geotiff(netcdf_path, output_folder):
    # Open the NetCDF file
    ds = nc.Dataset(netcdf_path)

    # Extract variables
    lon = ds.variables['lon'][:]
    lat = ds.variables['lat'][:]
    time = ds.variables['time']
    pr = ds.variables['pr']

    # Define the base date for the time dimension
    base_date = datetime(1900, 1, 1)
    
    # Determine the resolution of the grid
    lon_res = lon[1] - lon[0]
    lat_res = lat[1] - lat[0]

    # Create transform for GeoTIFF
    transform = from_origin(lon.min(), lat.max(), lon_res, lat_res)

    # Setup raster meta data
    meta = {
        'driver': 'GTiff',
        'dtype': 'float32',
        'nodata': None,
        'width': len(lon),
        'height': len(lat),
        'count': 1,
        'crs': rasterio.crs.CRS.from_epsg(4326),  # Assuming data is in WGS84
        'transform': transform
    }

    # Process each time step
    for i, t in enumerate(time):
        # Convert time to date. It seems the time t is in many cases just 0.
        # This is a problem as we don't have track of the time anymore.
        # Assume we increase every time by 1 day
        if float(t) > 0.:
            current_date = base_date + timedelta(days = float(t))
        else:
            current_date = current_date + timedelta(days = 1.)
        date_str = current_date.strftime('%Y-%m-%d')
        
        # Create a filename for each timestep
        filename = f"{output_folder}/pr_{date_str}.tif"
        
        # Read precipitation data for the current time step
        precipitation_data = np.flip(pr[i, :, :], axis = 0)
        
        # Write the data to a GeoTIFF file
        with rasterio.open(filename, 'w', **meta) as dst:
            dst.write(precipitation_data.filled(), 1)  # Use .filled() to handle masked arrays

    print("Conversion complete!")

Create a directory for the output data:

In [35]:
os.makedirs(outdir, exist_ok=True)

In [51]:
netcdf_to_geotiff(fname, outdir)

Conversion complete!


**Values after April 1995 seem to be all 0 !!!**